In [1]:
import json
import networkx as nx
import numpy as np

import functions

In [2]:
#function to read .json file of the input graph
def readPreferenceGraph(filePath) :
    with open(filePath) as file :
        data = json.load(file)

    G = nx.MultiDiGraph()

    for edge in data["edges"] :
        G.add_edge(edge["source"], edge["target"], opinion=edge["opinion"], questionItem=edge["question item"])
    
    return G

In [3]:
def createInteractionGraph(preferenceGraph, threshold, maxInteractions, maxTimestamp) :
    n = preferenceGraph.number_of_nodes()
    activitiesCount = 6

    interactionGraph = []
    interactionGraphIntensity = []
    
    degrees = [0] * n
   
        
    # list of possible peers to inteact for each person
    possiblePeers = functions.analyzeQuiz(prefGraph=preferenceGraph, \
                                          threshold=threshold, \
                                          numNodes=n)

    # ------------------------------------- #
    #    interactions at timetsamp t = 0    #
    # ------------------------------------- #
    # 1st step: interactions based on a scale-free network model with m=3
    interactionGraph.append(nx.barabasi_albert_graph(n=preferenceGraph.number_of_nodes(), \
                                                     m=3))

    # 2nd step: add random activities as weights
    for u, v in interactionGraph[0].edges() :
        interactionGraph[0][u][v]["interaction_index"] = np.random.choice(range(activitiesCount))
    
    # 3rd step: calculate intensity
    snapshotIntensity = functions.calculateIntensityForSnapshot(snapshotGraph=interactionGraph[0], 
                                                                numNodes=n)
    interactionGraphIntensity.append(snapshotIntensity)

    # 4th step: add snapshot degrees to the total number of degrees
    for node in range(n) :
        degrees[node] += interactionGraph[0].degree(node) 
    

    # -------------------------------------- #
    #    interactions at timetsamp t >= 1    #
    # -------------------------------------+ #
    for time in range(1, maxTimestamp + 1) :
        
        # 1st step: create new snapshot
        # 2nd step: add activities as weights
        snapshot = functions.createSnapshot(numNodes=n, \
                                            maxInteractions=maxInteractions, \
                                            degrees=degrees, \
                                            possiblePeers=possiblePeers, \
                                            activitiesCount=activitiesCount)
        interactionGraph.append(snapshot)
        
        # 3rd step: calculate intensity
        snapshotIntensity = functions.calculateIntensityForSnapshot(snapshotGraph=interactionGraph[time], \
                                                                    numNodes=n)
        interactionGraphIntensity.append(snapshotIntensity)
        
        # 4th step: add snapshot degrees to the total number of degrees
        for node in range(n) :
            degrees[node] += interactionGraph[time].degree(node) 

    
    # calculate indexes for each snapshot
    graphSequenceIndexes = []
    for time in range(maxTimestamp + 1) :
        snapshotIndexes = functions.calculateIndexes(prefGraph=preferenceGraph, \
                                                     snapshotGraph=interactionGraph[time], \
                                                     numNodes=n)
        graphSequenceIndexes.append(snapshotIndexes)


    return {
        "graph" : interactionGraph,
        "sequence intensity": interactionGraphIntensity,
        "individual": {
            "popularity": np.average([item["individual"]["popularity"] for item in graphSequenceIndexes]),
            "antipathy": np.average([item["individual"]["antipathy"] for item in graphSequenceIndexes]),
            "affective connection": np.average([item["individual"]["affective connection"] for item in graphSequenceIndexes]),
            "sociometric status": np.average([item["individual"]["sociometric status"] for item in graphSequenceIndexes]),
            "positive expansion": np.average([item["individual"]["positive expansion"] for item in graphSequenceIndexes]),
            "negative expansion": np.average([item["individual"]["negative expansion"] for item in graphSequenceIndexes]),
            "realistic perception": np.average([item["individual"]["realistic perception"] for item in graphSequenceIndexes])
        },
        "group": {
            "association": np.average([item["group"]["association"] for item in graphSequenceIndexes]),
            "dissociation": np.average([item["group"]["dissociation"] for item in graphSequenceIndexes]),
            "cohesion": np.average([item["group"]["cohesion"] for item in graphSequenceIndexes]),
            "social intensity": np.average([item["group"]["social intensity"] for item in graphSequenceIndexes]),
        }
    }

In [4]:
path = "./preferenceGraph.json"

options = {
    "threshold" : 0.4,
    "maxInteractions" : 200,
    "maxTimestamp" : 50
}

G = createInteractionGraph(preferenceGraph=readPreferenceGraph(path), **options)

print(f"Graph Sequence Intensity: {G['sequence intensity']}")
print("\nSnapshots added...")
for i in range(6) :
    print(f"at t = {i}: {G['graph'][i]}")

print("\n")
print(G["individual"])
print(G["group"])

Graph Sequence Intensity: [13831, 9255, 4528, 4552, 4689, 4337, 4171, 9411, 4545, 4525, 4787, 4295, 4358, 9143, 9572, 9084, 4349, 9099, 9116, 4410, 9319, 4406, 4260, 9513, 9350, 8899, 4315, 4284, 9224, 4364, 9357, 4035, 4168, 8915, 9118, 9386, 8551, 3687, 6619, 5341, 4159, 3203, 1260, 1895, 955, 257, 418, 238, 93, 119, 39]

Snapshots added...
at t = 0: Graph with 100 nodes and 291 edges
at t = 1: Graph with 100 nodes and 294 edges
at t = 2: Graph with 100 nodes and 198 edges
at t = 3: Graph with 100 nodes and 196 edges
at t = 4: Graph with 100 nodes and 198 edges
at t = 5: Graph with 100 nodes and 196 edges


{'popularity': 0.02706476530005942, 'antipathy': 0.014020598138245197, 'affective connection': 0.2298743007962957, 'sociometric status': 0.018510596157654984, 'positive expansion': 0.02706476530005942, 'negative expansion': 0.014020598138245197, 'realistic perception': 0.2248668894222527}
{'association': 0.0074549415725886315, 'dissociation': 0.001259655377302436, 'cohesion': 0.26

In [5]:
activities = ["to share content, chat in a social network platform",
            "to participate in debate/group presentation & discussion",
            "to play games",
            "to study in group or work on collaborative projects",
            "to be volunteers",
            "to be teammates in a sports team"]

for i,j,k in G["graph"][1].edges(data=True) :
    print(f"({i},{j}) => interaction: {activities[k['interaction_index']]}")

(0,85) => interaction: to share content, chat in a social network platform
(0,83) => interaction: to play games
(0,16) => interaction: to be volunteers
(0,11) => interaction: to study in group or work on collaborative projects
(1,30) => interaction: to share content, chat in a social network platform
(1,38) => interaction: to study in group or work on collaborative projects
(1,9) => interaction: to be teammates in a sports team
(1,11) => interaction: to be volunteers
(1,29) => interaction: to be volunteers
(2,87) => interaction: to share content, chat in a social network platform
(2,46) => interaction: to play games
(2,45) => interaction: to be volunteers
(2,12) => interaction: to be teammates in a sports team
(2,19) => interaction: to play games
(2,44) => interaction: to study in group or work on collaborative projects
(2,79) => interaction: to be volunteers
(2,96) => interaction: to play games
(3,21) => interaction: to share content, chat in a social network platform
(3,67) => intera